In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").appName("chapter7").getOrCreate()

24/01/02 13:48:00 WARN Utils: Your hostname, quangtn933.local resolves to a loopback address: 127.0.0.1; using 192.168.1.90 instead (on interface en0)
24/01/02 13:48:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/02 13:48:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/02 13:48:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Getting data

In [3]:
taxi_raw = spark.read.option("header","true").csv("taxidata")
taxi_raw.show(1, vertical=True)

-RECORD 0----------------------------------
 medallion          | 89D227B655E5C82AE... 
 hack_license       | BA96DE419E711691B... 
 vendor_id          | CMT                  
 rate_code          | 1                    
 store_and_fwd_flag | N                    
 pickup_datetime    | 2013-01-01 15:11:48  
 dropoff_datetime   | 2013-01-01 15:18:10  
 passenger_count    | 4                    
 trip_time_in_secs  | 382                  
 trip_distance      | 1.00                 
 pickup_longitude   | -73.978165           
 pickup_latitude    | 40.757977            
 dropoff_longitude  | -73.989838           
 dropoff_latitude   | 40.751171            
only showing top 1 row



In [4]:
taxi_raw.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In [5]:
from pyspark.sql import functions as fun

In [7]:
taxi_raw = taxi_raw.withColumn('pickup_datetime', fun.to_timestamp(fun.col('pickup_datetime'), "yyyy-MM-dd HH:mm:ss"))

In [8]:
taxi_raw

DataFrame[medallion: string, hack_license: string, vendor_id: string, rate_code: string, store_and_fwd_flag: string, pickup_datetime: timestamp, dropoff_datetime: string, passenger_count: string, trip_time_in_secs: string, trip_distance: string, pickup_longitude: string, pickup_latitude: string, dropoff_longitude: string, dropoff_latitude: string]

In [9]:
taxi_raw = taxi_raw.withColumn('dropoff_datetime', fun.to_timestamp(fun.col('dropoff_datetime'), "yyyy-MM-dd HH:mm:ss"))

In [10]:
taxi_raw.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In [11]:
taxi_raw.sort(fun.col("pickup_datetime").desc()).show(3, vertical=True)

24/01/02 13:56:39 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 14
CSV file: file:///Users/quangtn/Desktop/01_work/01_job/02_ml/PySpark/taxidata/trip_data_1.csv.zip


-RECORD 0----------------------------------
 medallion          | E3F00BB3F4E710383... 
 hack_license       | 10A2B96DE39865918... 
 vendor_id          | CMT                  
 rate_code          | 1                    
 store_and_fwd_flag | N                    
 pickup_datetime    | 2013-01-31 23:59:59  
 dropoff_datetime   | 2013-02-01 00:05:16  
 passenger_count    | 1                    
 trip_time_in_secs  | 317                  
 trip_distance      | 1.00                 
 pickup_longitude   | -73.990685           
 pickup_latitude    | 40.719158            
 dropoff_longitude  | -74.003288           
 dropoff_latitude   | 40.71521             
-RECORD 1----------------------------------
 medallion          | EA00A64CBDB68C77D... 
 hack_license       | 2045C77002FA0F2E0... 
 vendor_id          | CMT                  
 rate_code          | 1                    
 store_and_fwd_flag | N                    
 pickup_datetime    | 2013-01-31 23:59:59  
 dropoff_datetime   | 2013-02-01

In [12]:
geospatial_temporal_colnames = ["pickup_longitude", "pickup_latitude", "dropoff_longitude",
                                "dropoff_latitude", "pickup_datetime", "dropoff_datetime"]

In [13]:
taxi_raw.select([fun.count(fun.when(fun.isnull(c), c)).alias(c) for c in geospatial_temporal_colnames]).show()

24/01/02 14:02:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , , , , 
 Schema: pickup_datetime, dropoff_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude
Expected: pickup_datetime but found: 
CSV file: file:///Users/quangtn/Desktop/01_work/01_job/02_ml/PySpark/taxidata/trip_data_1.csv.zip


+----------------+---------------+-----------------+----------------+---------------+----------------+
|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|pickup_datetime|dropoff_datetime|
+----------------+---------------+-----------------+----------------+---------------+----------------+
|         7651368|        7651408|          7651502|         7651507|        7651423|         7651423|
+----------------+---------------+-----------------+----------------+---------------+----------------+



In [15]:
taxi_raw = taxi_raw.na.drop(subset=geospatial_temporal_colnames)

In [22]:
print("Count of zero dropoff, pickup latitude and longitude records")
taxi_raw.groupBy((fun.col("dropoff_longitude") == 0) | (fun.col("dropoff_latitude") == 0) |
                (fun.col("pickup_longitude") == 0) | (fun.col("pickup_latitude") == 0)).count().show()

Count of zero dropoff, pickup latitude and longitude records


24/01/02 14:11:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , , , , 
 Schema: pickup_datetime, dropoff_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude
Expected: pickup_datetime but found: 
CSV file: file:///Users/quangtn/Desktop/01_work/01_job/02_ml/PySpark/taxidata/trip_data_1.csv.zip


+----------------------------------------------------------------------------------------------------------+--------+
|((((dropoff_longitude = 0) OR (dropoff_latitude = 0)) OR (pickup_longitude = 0)) OR (pickup_latitude = 0))|   count|
+----------------------------------------------------------------------------------------------------------+--------+
|                                                                                                      NULL|     176|
|                                                                                                      true|  286433|
|                                                                                                     false|14489920|
+----------------------------------------------------------------------------------------------------------+--------+



## Analyze Geopandas

In [28]:
import geopandas as gdp

In [29]:
gdf = gdp.read_file("./nyc-boroughs.geojson")

In [30]:
gdf = gdf.to_crs(3857)

In [33]:
gdf.head(5)

,boroughCode,borough,@id,geometry
0,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-8243264.851 4948597.836, -8243206.4..."
1,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-8243557.877 4950251.047, -8243660.3..."
2,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-8255392.882 4959598.361, -8255451.0..."
3,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-8246794.198 4960600.673, -8246706.2..."
4,4,Queens,http://nyc.pediacities.com/Resource/Borough/Qu...,"POLYGON ((-8219461.925 4952778.732, -8219464.8..."


In [34]:
gdf['area'] = gdf.apply(lambda x: x['geometry'].area, axis=1)

In [35]:
gdf.head(5)

,boroughCode,borough,@id,geometry,area
0,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-8243264.851 4948597.836, -8243206.4...",1.997039e+04
1,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-8243557.877 4950251.047, -8243660.3...",8.180157e+04
2,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-8255392.882 4959598.361, -8255451.0...",1.248422e+05
3,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-8246794.198 4960600.673, -8246706.2...",2.616393e+08
4,4,Queens,http://nyc.pediacities.com/Resource/Borough/Qu...,"POLYGON ((-8219461.925 4952778.732, -8219464.8...",6.478373e+05


In [36]:
gdf = gdf.sort_values(by=['boroughCode', 'area'], ascending=[True, False])

In [37]:
gdf.head(5)

,boroughCode,borough,@id,geometry,area
72,1,Manhattan,http://nyc.pediacities.com/Resource/Borough/Ma...,"POLYGON ((-8229449.824 4994307.583, -8229437.5...",9.588180e+07
71,1,Manhattan,http://nyc.pediacities.com/Resource/Borough/Ma...,"POLYGON ((-8228885.652 4983011.624, -8228771.7...",3.809060e+06
51,1,Manhattan,http://nyc.pediacities.com/Resource/Borough/Ma...,"POLYGON ((-8239506.649 4967214.563, -8239357.0...",1.242664e+06
69,1,Manhattan,http://nyc.pediacities.com/Resource/Borough/Ma...,"POLYGON ((-8231163.561 4978335.590, -8231280.9...",1.019522e+06
73,1,Manhattan,http://nyc.pediacities.com/Resource/Borough/Ma...,"POLYGON ((-8227250.755 4994032.424, -8227372.2...",5.352348e+05


In [51]:
b_gdf = spark.sparkContext.broadcast(gdf)

In [52]:
def find_borough(latitude,longitude):
    mgdf = b_gdf.value.apply(lambda x: x['borough'] if \
                              x['geometry'].\
                              intersects(gdp.\
                                        points_from_xy(
                                            [longitude], \
                                            [latitude])[0]) \
                              else None, axis=1)
    idx = mgdf.first_valid_index()
    return mgdf.loc[idx] if idx is not None else None

In [53]:
from pyspark.sql.types import StringType

find_borough_udf = fun.udf(find_borough, StringType())

In [54]:
df_with_boroughs = taxi_raw.\
                    withColumn("dropoff_borough", \
                              find_borough_udf(
                                fun.col("dropoff_latitude"),\
                                fun.col('dropoff_longitude')))

In [ ]:
df_with_boroughs.groupBy(fun.col("dropoff_borough")).count().show()

In [ ]:
from pyspark.sql import Window

window_spec = Window.partitionBy("hack_license").orderBy(fun.col("hack_license"), fun.col("pickup_datetime"))

In [ ]:
df_with_borough_durations = df_with_boroughs.withColumn("trip_time_difference", fun.col("pickup_datetime")-fun.lag(fun.col("pickup_datetime"), 1).over(window_spec)).show(50, vertical=True)

In [ ]:
df_with_borough_durations.selectExpr("floor(seconds / 3600) as hours").groupBy("hours").count().sort("hours").show()

In [ ]:
df_with_borough_durations.where("seconds > 0 AND seconds < 60*60*4").groupBy("borough").agg(avg("seconds"), stddev("seconds")).show()